In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def geturl(webpage):
    url = webpage
    resp = requests.get(url)
    resp.encoding = 'utf-8'
    #print(resp.text)
    return resp

In [59]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd



#Yahoo電影
url = 'https://movies.yahoo.com.tw/movie_intheaters.html' #目標位置

def check_req_url(url): #測試請求網址是否請求成功
    resp = requests.get(url) #請求網址
    #print(resp.status_code) #錯誤時404,成功時200
    if resp.status_code != 200:  #如果請求失敗
        print('Invalid url:', resp.url) #印出請求失敗的網址
        return "fail" #回傳失敗提示訊息
    else:
        return resp.text #回傳請求成功的html文字

def get_week_new_movies(webpage): #抓取電影資訊
    soup = BeautifulSoup(webpage, 'html5lib') #網頁解析
    movies = [] #域設電影資訊存這裡 
   
    #抓取<div class="release_info_text"></div>內文字
    rows = soup.find_all('div', 'release_info_text')
    data_movie = {}
    #print(rows)
    for row in rows:
        data_movie = dict() #存成{"key":"value"}格式
        #電影名稱
        data_movie['ch_name'] = row.find('div', 'release_movie_name').a.text.strip()
        #英文名稱
        data_movie['english_name'] = row.find('div', 'release_movie_name').find('div', 'en').a.text.strip()
        #電影分類
        data_movie['type'] = soup.title.text.replace('類電影 - Yahoo奇摩電影', '')
        #電影介紹
        data_movie['info'] = row.find('div', class_='release_text').find('span').text.replace(' ', '').replace('\n', '').replace('\r\xa0', '').replace('\r', '')
        #上映日期 只抓日期 "上映日期：2020-11-20" -> match.group(0): "2020-11-20"
        data_movie['release_date'] = get_date(row.find('div', 'release_movie_time').text)
        #電影ID
        data_movie['movie_id'] = get_movie_id(row.find('div', 'release_movie_name').a['href'])
       
        movies.append(data_movie) #再被取代前先存入for 外面的movies=[]
        
        
    return movies

def get_movie_id(url):
    
    try:
        movie_id = url.split('-')[-1]
        #print(movie_id)
    except:
        movie_id = url
    return movie_id


def get_date(date_str):
    # e.x. "上映日期：2017-03-23" -> match.group(0): "2020-11-20"
    #記得import re
    #1.\d找到數字
    #2.要找一整組的所以\d+，+代表1個以上
    #3.有可能有'-'

    pattern = '\d+-\d+-\d+'
    match = re.search(pattern, date_str)
    #print(match)
    #print(match.group(0))
    
    if match is None:
        return date_str
    else:
        return match.group(0)

if __name__ == '__main__':

    movie = []
    for genreID in range (1,20):
        genre_url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=%s' % genreID
        for page in range(1,442):
            url = genre_url+'&page=%s' % page
            webpage = check_req_url(url)

            if webpage:
                movies = get_week_new_movies(webpage)
                movie.append(movies)
            
    print(pd.DataFrame(movie))
    pd.DataFrame(movie).to_csv("./movie_id.csv",encoding="utf-8-sig",index=False)
       

                                                      0  \
0     {'ch_name': '小小兵2：格魯的崛起', 'english_name': 'Min...   
1     {'ch_name': '逃出摩加迪休', 'english_name': 'Escape ...   
2     {'ch_name': '宇宙特攻隊', 'english_name': 'Cosmobal...   
3     {'ch_name': '永生戰', 'english_name': 'Seobok', '...   
4     {'ch_name': '迴路追殺令', 'english_name': 'Boss Lev...   
...                                                 ...   
8374                                               None   
8375                                               None   
8376                                               None   
8377                                               None   
8378                                               None   

                                                      1  \
0     {'ch_name': '蝙蝠俠', 'english_name': 'The Batman...   
1     {'ch_name': '密弒遊戲2：勝者危亡', 'english_name': 'Esc...   
2     {'ch_name': '黑寡婦', 'english_name': 'Black Wido...   
3     {'ch_name': '真人快打', 'english_name': 'Mortal Ko...